In [1]:
import numpy as np
import pandas as pd

# Integrated Data

In [2]:
data = pd.read_csv("input/Integrated_data.csv", low_memory = False)

# Load up the list of variables we want to aggregate

In [3]:
variables = pd.read_csv("input/IVS_Variable_List.csv")

In [4]:
variables

,Included,Name,Label,Last EVS,Last WVS,Unnamed: 5
0,1,A001,Important in life: Family,EVS 2008,WVS6 (2010-2012),NaN
1,1,A002,Important in life: Friends,EVS 2008,WVS6 (2010-2012),NaN
2,0,A003,Important in life: Leisure time,EVS 2008,WVS6 (2010-2012),NaN
3,0,A004,Important in life: Politics,EVS 2008,WVS6 (2010-2012),NaN
4,1,A005,Important in life: Work,EVS 2008,WVS6 (2010-2012),NaN
...,...,...,...,...,...,...
1421,0,w007,Spouse/partner had/has how many employees,EVS 2008,NaN,NaN
1422,0,w008,Does spouse/partner supervise someone,EVS 2008,NaN,NaN
1423,0,w009,How many people does she/he supervise,EVS 2008,NaN,NaN
1424,0,w010,Spouse/partner experienced unemployment longer...,EVS 2008,NaN,NaN


In [5]:
variables_to_keep = variables[variables.Included == 1].Name

# Slice the dataframe and preprocess some columns

In [6]:
data_subset = data.loc[: , data.columns.isin(variables_to_keep)].copy()

In [7]:
WVS_wave = pd.read_csv("input/WVS_wave.csv", index_col = "code").wave.to_dict()
EVS_wave = pd.read_csv("input/EVS_wave.csv", index_col = "code").wave.to_dict()

WVS_wave[-4] = ""
EVS_wave[-4] = ""

In [8]:
for key in WVS_wave:
    data_subset.loc[data_subset['S002'] == key, ['S002']] = WVS_wave[key]
    
for key in EVS_wave:
    data_subset.loc[data_subset['S002EVS'] == key, ['S002EVS']] = EVS_wave[key]
    
data_subset.S002 += data_subset.S002EVS

data_subset.drop(['S002EVS'], axis = 1, inplace = True)
data_subset.rename(columns={'S002':'Wave'}, inplace=True)


In [9]:
data_subset.loc[data_subset['S001'] == 1, ['S001']] = "EVS"
data_subset.loc[data_subset['S001'] == 2, ['S001']] = "WVS"

data_subset.rename(columns={'S001':'Survey'}, inplace=True)

In [10]:
data_subset.rename(columns={'S003':'Country Numeric'}, inplace=True)
data_subset.rename(columns={'X048':'Region Numeric'}, inplace=True)
data_subset.rename(columns={'S017':'Weight'}, inplace=True)


In [11]:
data_subset

,Survey,Wave,Country Numeric,Weight,A001,A002,A005,A009,A029,A032,...,X026,X028,X036,X045,X047,Region Numeric,X049,X051,X052,X053
0,WVS,1981-1984,32,0.926626,-4,-4,-4,4,0,1,...,-2,4,33,-2,-4,-4,-4,-4,-4,-4
1,WVS,1981-1984,32,0.926626,-4,-4,-4,4,0,0,...,-2,4,-3,2,-4,-4,-4,-4,-4,-4
2,WVS,1981-1984,32,0.926626,-4,-4,-4,3,0,0,...,-2,1,33,2,-4,-4,-4,-4,-4,-4
3,WVS,1981-1984,32,1.352874,-4,-4,-4,2,1,1,...,1,7,-3,2,-4,-4,-4,-4,-4,-4
4,WVS,1981-1984,32,0.926626,-4,-4,-4,2,1,0,...,-2,5,-3,3,-4,-4,-4,-4,-4,-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513524,EVS,2008-2010,915,0.782164,1,1,1,2,1,1,...,-4,1,-4,-4,-4,-4,-5,-4,-4,-4
513525,EVS,2008-2010,915,0.887335,1,2,2,4,0,1,...,-4,1,-4,-4,-4,-4,-5,-4,-4,-4
513526,EVS,2008-2010,915,0.782164,1,2,2,2,1,0,...,-4,1,-4,-4,-4,-4,-5,-4,-4,-4
513527,EVS,2008-2010,915,1.201044,1,1,1,3,0,1,...,-4,1,-4,-4,-4,-4,-5,-4,-4,-4


## One-hot encoding

In [12]:
not_to_encode = ["Survey", "Wave", "Country Numeric", "Region Numeric", "Weight"] # are the variables we want to groupby and the weights (S017)

In [13]:
one_hot = pd.get_dummies(data=data_subset.loc[: , [c for c in data_subset.columns if c not in not_to_encode] ], 
                           columns=[c for c in data_subset.columns if c not in not_to_encode])

In [14]:
one_hot

,A001_-5,A001_-4,A001_-2,A001_-1,A001_1,A001_2,A001_3,A001_4,A002_-5,A002_-4,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
0,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513524,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
513525,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
513526,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
513527,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Multiply by the weights

In [15]:
weighted_one_hot = one_hot.mul(data_subset["Weight"], axis=0).copy()

In [16]:
weighted_one_hot

,A001_-5,A001_-4,A001_-2,A001_-1,A001_1,A001_2,A001_3,A001_4,A002_-5,A002_-4,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
0,0.0,0.926626,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.926626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.926626,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.926626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.926626,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.926626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.352874,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.352874,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.926626,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.926626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513524,0.0,0.000000,0.0,0.0,0.782164,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
513525,0.0,0.000000,0.0,0.0,0.887335,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
513526,0.0,0.000000,0.0,0.0,0.782164,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
513527,0.0,0.000000,0.0,0.0,1.201044,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Groupby Country

### For country statistics we can correct the numbers using S007

In [17]:
weighted_one_hot["Survey"] = data_subset["Survey"]
weighted_one_hot["Wave"] = data_subset["Wave"]
weighted_one_hot["Country Numeric"] = data_subset["Country Numeric"]

In [18]:
grouped_by_country = weighted_one_hot.groupby(["Survey", "Wave", "Country Numeric"]).mean().copy()

In [19]:
grouped_by_country["Sample Size"] = weighted_one_hot.groupby(["Survey", "Wave", "Country Numeric"]).size()

In [20]:
grouped_by_country

A001_-5  A001_-4   A001_-2  A001_-1  \
Survey Wave      Country Numeric                                        
EVS    1981-1984 56                   0.0      1.0  0.000000    0.000   
                 124                  0.0      1.0  0.000000    0.000   
                 208                  0.0      1.0  0.000000    0.000   
                 250                  0.0      1.0  0.000000    0.000   
                 276                  0.0      1.0  0.000000    0.000   
...                                   ...      ...       ...      ...   
WVS    2010-2014 818                  0.0      0.0  0.000000    0.000   
                 840                  0.0      0.0  0.003548    0.000   
                 858                  0.0      0.0  0.000000    0.001   
                 860                  0.0      0.0  0.000000    0.000   
                 887                  0.0      0.0  0.000000    0.006   

                                    A001_1    A001_2    A001_3    A001_4  \
Survey Wave      Country Numeric                                           
EVS    1981-1984 56               0.000000  0.000000  0.000000  0.000000   
                 124              0.000000  0.000000  0.000000  0.000000   
                 208              0.000000  0.000000  0.000000  0.000000   
                 250              0.000000  0.000000  0.000000  0.000000   
                 276              0.000000  0.000000  0.000000  0.000000   
...                                    ...       ...       ...       ...   
WVS    2010-2014 818              0.973568  0.021512  0.004920  0.000000   
                 840              0.909207  0.073387  0.007944  0.005945   
                 858              0.887000  0.099000  0.006000  0.007000   
                 860              0.975333  0.021333  0.003333  0.000000   
                 887              0.958000  0.029000  0.006000  0.001000   

                                  A002_-5  A002_-4  ...    X053_2    X053_3  \
Survey Wave      Country Numeric                    ...                       
EVS    1981-1984 56                   0.0      1.0  ...  0.000000  0.000000   
                 124                  0.0      1.0  ...  0.000000  0.000000   
                 208                  0.0      1.0  ...  0.000000  0.000000   
                 250                  0.0      1.0  ...  0.000000  0.000000   
                 276                  0.0      1.0  ...  0.000000  0.000000   
...                                   ...      ...  ...       ...       ...   
WVS    2010-2014 818                  0.0      0.0  ...  0.049620  0.073058   
                 840                  0.0      0.0  ...  0.043508  0.054001   
                 858                  0.0      0.0  ...  0.104000  0.062000   
                 860                  0.0      0.0  ...  0.094667  0.078667   
                 887                  0.0      0.0  ...  0.073000  0.034000   

                                    X053_4    X053_5    X053_6    X053_7  \
Survey Wave      Country Numeric                                           
EVS    1981-1984 56               0.000000  0.000000  0.000000  0.000000   
                 124              0.000000  0.000000  0.000000  0.000000   
                 208              0.000000  0.000000  0.000000  0.000000   
                 250              0.000000  0.000000  0.000000  0.000000   
                 276              0.000000  0.000000  0.000000  0.000000   
...                                    ...       ...       ...       ...   
WVS    2010-2014 818              0.045724  0.052922  0.037831  0.046540   
                 840              0.043248  0.152357  0.101210  0.100090   
                 858              0.057000  0.130000  0.037000  0.040000   
                 860              0.075333  0.078667  0.061333  0.031333   
                 887              0.014000  0.035000  0.020000  0.021000   

                                    X053_8    X053_9   X053_10  Sample Size  


### Lets add the country names

In [21]:
country_ISO = pd.read_csv("input/ISO_3611.csv", index_col = "numeric")

In [22]:
country_ISO

,Country,alpha-2,alpha-3
numeric,,,
4,Afghanistan,AF,AFG
8,Albania,AL,ALB
12,Algeria,DZ,DZA
16,American Samoa,AS,ASM
20,Andorra,AD,AND
...,...,...,...
101,Republika Srpska,NaN,NaN
197,Northern Cyprus,NaN,NaN
909,North Ireland,NaN,Nan


In [23]:
names = []
ISO3 = []

for idx in grouped_by_country.index:
    names.append(country_ISO.loc[idx[2], "Country"])
    ISO3.append(country_ISO.loc[idx[2], "alpha-3"])

In [24]:
grouped_by_country["Country Name"] = names
grouped_by_country["Country ISO3"] = ISO3

In [25]:
grouped_by_country

A001_-5  A001_-4   A001_-2  A001_-1  \
Survey Wave      Country Numeric                                        
EVS    1981-1984 56                   0.0      1.0  0.000000    0.000   
                 124                  0.0      1.0  0.000000    0.000   
                 208                  0.0      1.0  0.000000    0.000   
                 250                  0.0      1.0  0.000000    0.000   
                 276                  0.0      1.0  0.000000    0.000   
...                                   ...      ...       ...      ...   
WVS    2010-2014 818                  0.0      0.0  0.000000    0.000   
                 840                  0.0      0.0  0.003548    0.000   
                 858                  0.0      0.0  0.000000    0.001   
                 860                  0.0      0.0  0.000000    0.000   
                 887                  0.0      0.0  0.000000    0.006   

                                    A001_1    A001_2    A001_3    A001_4  \
Survey Wave      Country Numeric                                           
EVS    1981-1984 56               0.000000  0.000000  0.000000  0.000000   
                 124              0.000000  0.000000  0.000000  0.000000   
                 208              0.000000  0.000000  0.000000  0.000000   
                 250              0.000000  0.000000  0.000000  0.000000   
                 276              0.000000  0.000000  0.000000  0.000000   
...                                    ...       ...       ...       ...   
WVS    2010-2014 818              0.973568  0.021512  0.004920  0.000000   
                 840              0.909207  0.073387  0.007944  0.005945   
                 858              0.887000  0.099000  0.006000  0.007000   
                 860              0.975333  0.021333  0.003333  0.000000   
                 887              0.958000  0.029000  0.006000  0.001000   

                                  A002_-5  A002_-4  ...    X053_4    X053_5  \
Survey Wave      Country Numeric                    ...                       
EVS    1981-1984 56                   0.0      1.0  ...  0.000000  0.000000   
                 124                  0.0      1.0  ...  0.000000  0.000000   
                 208                  0.0      1.0  ...  0.000000  0.000000   
                 250                  0.0      1.0  ...  0.000000  0.000000   
                 276                  0.0      1.0  ...  0.000000  0.000000   
...                                   ...      ...  ...       ...       ...   
WVS    2010-2014 818                  0.0      0.0  ...  0.045724  0.052922   
                 840                  0.0      0.0  ...  0.043248  0.152357   
                 858                  0.0      0.0  ...  0.057000  0.130000   
                 860                  0.0      0.0  ...  0.075333  0.078667   
                 887                  0.0      0.0  ...  0.014000  0.035000   

                                    X053_6    X053_7    X053_8    X053_9  \
Survey Wave      Country Numeric                                           
EVS    1981-1984 56               0.000000  0.000000  0.000000  0.000000   
                 124              0.000000  0.000000  0.000000  0.000000   
                 208              0.000000  0.000000  0.000000  0.000000   
                 250              0.000000  0.000000  0.000000  0.000000   
                 276              0.000000  0.000000  0.000000  0.000000   
...                                    ...       ...       ...       ...   
WVS    2010-2014 818              0.037831  0.046540  0.063399  0.017063   
                 840              0.101210  0.100090  0.122997  0.110341   
                 858              0.037000  0.040000  0.065000  0.045000   
                 860              0.061333  0.031333  0.058000  0.042000   
                 887              0.020000  0.021000  0.041000  0.025000   

                                   X053_10  Sample Size  \
Survey Wave      Co

### Flat the index

In [26]:
grouped_by_country = grouped_by_country.reset_index()

In [27]:
grouped_by_country

,Survey,Wave,Country Numeric,A001_-5,A001_-4,A001_-2,A001_-1,A001_1,A001_2,A001_3,...,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10,Sample Size,Country Name,Country ISO3
0,EVS,1981-1984,56,0.0,1.0,0.000000,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1145,Belgium,BEL
1,EVS,1981-1984,124,0.0,1.0,0.000000,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1254,Canada,CAN
2,EVS,1981-1984,208,0.0,1.0,0.000000,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1182,Denmark,DNK
3,EVS,1981-1984,250,0.0,1.0,0.000000,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1200,France,FRA
4,EVS,1981-1984,276,0.0,1.0,0.000000,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1305,Germany,DEU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,WVS,2010-2014,818,0.0,0.0,0.000000,0.000,0.973568,0.021512,0.004920,...,0.045724,0.052922,0.037831,0.046540,0.063399,0.017063,0.032774,1523,Egypt,EGY
362,WVS,2010-2014,840,0.0,0.0,0.003548,0.000,0.909207,0.073387,0.007944,...,0.043248,0.152357,0.101210,0.100090,0.122997,0.110341,0.136770,2232,United States of America (the),USA
363,WVS,2010-2014,858,0.0,0.0,0.000000,0.001,0.887000,0.099000,0.006000,...,0.057000,0.130000,0.037000,0.040000,0.065000,0.045000,0.064000,1000,Uruguay,URY
364,WVS,2010-2014,860,0.0,0.0,0.000000,0.000,0.975333,0.021333,0.003333,...,0.075333,0.078667,0.061333,0.031333,0.058000,0.042000,0.094667,1500,Uzbekistan,UZB


### Move the last columns to the front

In [28]:
columns = list(grouped_by_country.columns)

In [29]:
grouped_by_country = grouped_by_country[columns[:3] + columns[-2:] + [columns[-3]] + columns[3:-3]]

In [30]:
grouped_by_country

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Sample Size,A001_-5,A001_-4,A001_-2,A001_-1,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
0,EVS,1981-1984,56,Belgium,BEL,1145,0.0,1.0,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,EVS,1981-1984,124,Canada,CAN,1254,0.0,1.0,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,EVS,1981-1984,208,Denmark,DNK,1182,0.0,1.0,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,EVS,1981-1984,250,France,FRA,1200,0.0,1.0,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,EVS,1981-1984,276,Germany,DEU,1305,0.0,1.0,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,WVS,2010-2014,818,Egypt,EGY,1523,0.0,0.0,0.000000,0.000,...,0.080814,0.049620,0.073058,0.045724,0.052922,0.037831,0.046540,0.063399,0.017063,0.032774
362,WVS,2010-2014,840,United States of America (the),USA,2232,0.0,0.0,0.003548,0.000,...,0.106743,0.043508,0.054001,0.043248,0.152357,0.101210,0.100090,0.122997,0.110341,0.136770
363,WVS,2010-2014,858,Uruguay,URY,1000,0.0,0.0,0.000000,0.001,...,0.329000,0.104000,0.062000,0.057000,0.130000,0.037000,0.040000,0.065000,0.045000,0.064000
364,WVS,2010-2014,860,Uzbekistan,UZB,1500,0.0,0.0,0.000000,0.000,...,0.237333,0.094667,0.078667,0.075333,0.078667,0.061333,0.031333,0.058000,0.042000,0.094667


### Save to csv

In [31]:
grouped_by_country.to_csv("out/IVS_grouped_by_country.csv", index = False, encoding = 'utf-16')

# Groupby Country and Region

### For regional aggregation we can not use the weighted answers

In [32]:
one_hot["Survey"] = data_subset["Survey"]
one_hot["Wave"] = data_subset["Wave"]
one_hot["Country Numeric"] = data_subset["Country Numeric"]
one_hot["Region Numeric"] = data_subset["Region Numeric"]

In [33]:
grouped_by_country_region = one_hot.groupby(["Survey", "Wave", "Country Numeric", "Region Numeric"]).mean().copy()

In [34]:
grouped_by_country_region["Sample Size"] = one_hot.groupby(["Survey", "Wave", "Country Numeric", "Region Numeric"]).size()

In [35]:
grouped_by_country_region

A001_-5  A001_-4   A001_-2  \
Survey Wave      Country Numeric Region Numeric                               
EVS    1981-1984 56               56021              0.0      1.0  0.000000   
                                  56022              0.0      1.0  0.000000   
                                  56023              0.0      1.0  0.000000   
                                  56024              0.0      1.0  0.000000   
                                  56025              0.0      1.0  0.000000   
...                                                  ...      ...       ...   
WVS    2010-2014 818             -4                  0.0      0.0  0.000000   
                 840             -4                  0.0      0.0  0.004928   
                 858             -4                  0.0      0.0  0.000000   
                 860             -4                  0.0      0.0  0.000000   
                 887             -4                  0.0      0.0  0.000000   

                                                 A001_-1    A001_1    A001_2  \
Survey Wave      Country Numeric Region Numeric                                
EVS    1981-1984 56               56021            0.000  0.000000  0.000000   
                                  56022            0.000  0.000000  0.000000   
                                  56023            0.000  0.000000  0.000000   
                                  56024            0.000  0.000000  0.000000   
                                  56025            0.000  0.000000  0.000000   
...                                                  ...       ...       ...   
WVS    2010-2014 818             -4                0.000  0.963230  0.034143   
                 840             -4                0.000  0.915323  0.065860   
                 858             -4                0.001  0.887000  0.099000   
                 860             -4                0.000  0.975333  0.021333   
                 887             -4                0.006  0.958000  0.029000   

                                                   A001_3    A001_4  A002_-5  \
Survey Wave      Country Numeric Region Numeric                                
EVS    1981-1984 56               56021          0.000000  0.000000      0.0   
                                  56022          0.000000  0.000000      0.0   
                                  56023          0.000000  0.000000      0.0   
                                  56024          0.000000  0.000000      0.0   
                                  56025          0.000000  0.000000      0.0   
...                                                   ...       ...      ...   
WVS    2010-2014 818             -4              0.002626  0.000000      0.0   
                 840             -4              0.008513  0.005376      0.0   
                 858             -4              0.006000  0.007000      0.0   
                 860             -4              0.003333  0.000000      0.0   
                 887             -4              0.006000  0.001000      0.0   

                                                 A002_-4  ...    X053_2  \
Survey Wave      Country Numeric Region Numeric           ...             
EVS    1981-1984 56               56021              1.0  ...  0.000000   
                                  56022              1.0  ...  0.000000   
                                  56023              1.0  ...  0.000000   
                                  56024              1.0  ...  0.000000   
                                  56025              1.0  ...  0.000000   
...                                                  ...  ...       ...   
WVS    2010-2014 818             -4                  0.0  ...  0.036113   
                 840             -4                  0.0  ...  0.040323   
                 858             -4                  0.0  ...  0.104000   
                 860             -4                  0.0  ...  0.094667   
                 887             -4    

### Add the country and region names

In [36]:
region_codes = pd.read_csv("input/Region_codes.csv", index_col="Code")

In [37]:
region_name = []
r_names = []
r_ISO3 = []


for idx in grouped_by_country_region.index:
    region_name.append(region_codes.loc[idx[3], "Region"])
    r_names.append(country_ISO.loc[idx[2], "Country"])
    r_ISO3.append(country_ISO.loc[idx[2], "alpha-3"])
    

In [38]:
grouped_by_country_region["Country Name"] = r_names
grouped_by_country_region["Country ISO3"] = r_ISO3
grouped_by_country_region["Region Name"] = region_name

### Flat the index

In [39]:
grouped_by_country_region = grouped_by_country_region.reset_index()
grouped_by_country_region

,Survey,Wave,Country Numeric,Region Numeric,A001_-5,A001_-4,A001_-2,A001_-1,A001_1,A001_2,...,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10,Sample Size,Country Name,Country ISO3,Region Name
0,EVS,1981-1984,56,56021,0.0,1.0,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6,Belgium,BEL,BE: unknown
1,EVS,1981-1984,56,56022,0.0,1.0,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,175,Belgium,BEL,BE: unknown
2,EVS,1981-1984,56,56023,0.0,1.0,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,109,Belgium,BEL,BE: unknown
3,EVS,1981-1984,56,56024,0.0,1.0,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,139,Belgium,BEL,BE: unknown
4,EVS,1981-1984,56,56025,0.0,1.0,0.000000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,163,Belgium,BEL,BE: unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,WVS,2010-2014,818,-4,0.0,0.0,0.000000,0.000,0.963230,0.034143,...,0.045305,0.021668,0.034800,0.037426,0.013789,0.019698,1523,Egypt,EGY,Not asked
1153,WVS,2010-2014,840,-4,0.0,0.0,0.004928,0.000,0.915323,0.065860,...,0.140233,0.101254,0.099014,0.130824,0.125000,0.139785,2232,United States of America (the),USA,Not asked
1154,WVS,2010-2014,858,-4,0.0,0.0,0.000000,0.001,0.887000,0.099000,...,0.130000,0.037000,0.040000,0.065000,0.045000,0.064000,1000,Uruguay,URY,Not asked
1155,WVS,2010-2014,860,-4,0.0,0.0,0.000000,0.000,0.975333,0.021333,...,0.078667,0.061333,0.031333,0.058000,0.042000,0.094667,1500,Uzbekistan,UZB,Not asked


### Move the last column to the front

In [40]:
columns_r = list(grouped_by_country_region.columns)

In [41]:
grouped_by_country_region = grouped_by_country_region[columns_r[:3] + columns_r[-2:-1]
                                                      + [columns_r[3]]+ [columns_r[-1]]
                                                      + [columns_r[-4]]+ columns_r[4:-4]]

In [42]:
grouped_by_country_region

,Survey,Wave,Country Numeric,Country ISO3,Region Numeric,Region Name,Sample Size,A001_-5,A001_-4,A001_-2,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
0,EVS,1981-1984,56,BEL,56021,BE: unknown,6,0.0,1.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,EVS,1981-1984,56,BEL,56022,BE: unknown,175,0.0,1.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,EVS,1981-1984,56,BEL,56023,BE: unknown,109,0.0,1.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,EVS,1981-1984,56,BEL,56024,BE: unknown,139,0.0,1.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,EVS,1981-1984,56,BEL,56025,BE: unknown,163,0.0,1.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,WVS,2010-2014,818,EGY,-4,Not asked,1523,0.0,0.0,0.000000,...,0.063033,0.036113,0.057124,0.033487,0.045305,0.021668,0.034800,0.037426,0.013789,0.019698
1153,WVS,2010-2014,840,USA,-4,Not asked,2232,0.0,0.0,0.004928,...,0.097670,0.040323,0.053315,0.042115,0.140233,0.101254,0.099014,0.130824,0.125000,0.139785
1154,WVS,2010-2014,858,URY,-4,Not asked,1000,0.0,0.0,0.000000,...,0.329000,0.104000,0.062000,0.057000,0.130000,0.037000,0.040000,0.065000,0.045000,0.064000
1155,WVS,2010-2014,860,UZB,-4,Not asked,1500,0.0,0.0,0.000000,...,0.237333,0.094667,0.078667,0.075333,0.078667,0.061333,0.031333,0.058000,0.042000,0.094667


### Save to csv

In [43]:
grouped_by_country_region.to_csv("out/IVS_grouped_by_country_region.csv", index = False, encoding = 'utf-16')